In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [3]:
from langchain_openai import ChatOpenAI

# Simple Chain

In [4]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [5]:
chain = prompt | model | output_parser

In [6]:
chain.invoke({"topic": "bears"})

"Why did the bear break up with his girlfriend? \nBecause he couldn't bear the relationship anymore!"

# Complex chain

In [9]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

In [10]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)

/home/david/anaconda3/envs/r_langchain/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [11]:
vectorstore

In [13]:
retriever = vectorstore.as_retriever()

In [15]:
retriever.invoke("where did harrison work?")

[Document(metadata={}, page_content='harrison worked at kensho'),
 Document(metadata={}, page_content='bears like to eat honey')]

In [16]:
retriever.invoke("what do bears like to eat?")

[Document(metadata={}, page_content='bears like to eat honey'),
 Document(metadata={}, page_content='harrison worked at kensho')]

In [17]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [18]:
from langchain.schema.runnable import RunnableMap


In [19]:
RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
})

{
  context: RunnableLambda(...),
  question: RunnableLambda(...)
}

In [20]:
run_map = RunnableMap({
    "context": lambda x: retriever.invoke(x["question"]),
    "question": lambda x: x["question"]
})

In [23]:
prompt_chain = run_map | prompt
chat_prompt_value = prompt_chain.invoke({"question": "where did harrison work?"})

In [24]:
chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(metadata={}, page_content='harrison worked at kensho'), Document(metadata={}, page_content='bears like to eat honey')]\n\nQuestion: where did harrison work?\n", additional_kwargs={}, response_metadata={})])

In [25]:
chat_prompt_value.messages

[HumanMessage(content="Answer the question based only on the following context:\n[Document(metadata={}, page_content='harrison worked at kensho'), Document(metadata={}, page_content='bears like to eat honey')]\n\nQuestion: where did harrison work?\n", additional_kwargs={}, response_metadata={})]

In [26]:
message = chat_prompt_value.messages[0]
print(message.content)

Answer the question based only on the following context:
[Document(metadata={}, page_content='harrison worked at kensho'), Document(metadata={}, page_content='bears like to eat honey')]

Question: where did harrison work?



In [21]:
chain = run_map | prompt | model | output_parser

In [27]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

# Bind

### Open AI Functions

In [29]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [30]:
# using the 'bind' - modifying parameters at runtime
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [31]:
runnable = prompt | model

In [32]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 64, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-f26220d3-30e7-46ef-a374-421110edb311-0', usage_metadata={'input_tokens': 64, 'output_tokens': 17, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Multiple Functions

In [33]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
        {
      "name": "sports_search",
      "description": "Search for news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [34]:
model = model.bind(functions=functions)

In [35]:
runnable = prompt | model

In [36]:
aimessage = runnable.invoke({"input": "how did the patriots do yesterday?"})


In [37]:
aimessage

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"patriots"}', 'name': 'sports_search'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 99, 'total_tokens': 118, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-fb71eab6-df60-4860-82d8-2e3a9aa5052c-0', usage_metadata={'input_tokens': 99, 'output_tokens': 19, 'total_tokens': 118, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [38]:
aimessage.additional_kwargs

{'function_call': {'arguments': '{"team_name":"patriots"}',
  'name': 'sports_search'},
 'refusal': None}

### Fallbacks

In [41]:
from langchain.llms import OpenAI
import json

gpt_model = "gpt-4o-mini"

In [ ]:
simple_model = ChatOpenAI(
    temperature=0, 
    max_tokens=1000, 
    model=gpt_model
)

In [43]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"
simple_chain = simple_model | json.loads

In [44]:
response = simple_model.invoke(challenge)

In [45]:
response

AIMessage(content='Here is a JSON blob containing three poems, each represented as a JSON object with a title, author, and first line:\n\n```json\n{\n  "poems": [\n    {\n      "title": "Whispers of the Dawn",\n      "author": "Emily Carter",\n      "first_line": "In the hush of morning light, dreams take flight."\n    },\n    {\n      "title": "The Dance of Leaves",\n      "author": "Michael Thompson",\n      "first_line": "Beneath the ancient oak, the leaves begin to sway."\n    },\n    {\n      "title": "Echoes of the Sea",\n      "author": "Sophia Lee",\n      "first_line": "Waves crash softly, singing secrets to the shore."\n    }\n  ]\n}\n```\n\nThis JSON structure contains three distinct poems, each with its own title, author, and first line.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 183, 'prompt_tokens': 31, 'total_tokens': 214, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_token

In [48]:
chain = simple_model | StrOutputParser()

In [49]:
response = chain.invoke(challenge)


In [50]:
response

'Here is a JSON blob containing three poems, each represented as a JSON object with a title, author, and first line:\n\n```json\n{\n  "poems": [\n    {\n      "title": "Whispers of the Dawn",\n      "author": "Emily Carter",\n      "first_line": "In the hush of morning light, dreams take flight."\n    },\n    {\n      "title": "The Ocean\'s Embrace",\n      "author": "James Holloway",\n      "first_line": "Waves crash softly, a lullaby of the sea."\n    },\n    {\n      "title": "Autumn\'s Lament",\n      "author": "Sophia Lee",\n      "first_line": "Leaves fall like whispers, secrets of the past."\n    }\n  ]\n}\n```\n\nFeel free to modify any part of the JSON structure or the content as needed!'

In [ ]:
final_chain = simple_chain.with_fallbacks([chain])
final_chain.invoke(challenge)


## Interface

In [51]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [52]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

["Why don't bears wear shoes? \n\nBecause they have bear feet!",
 'Why are frogs so happy?\n\nBecause they eat whatever bugs them!']

In [53]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 do
 bears
 have
 hairy
 coats
?


F
ur
 protection
!



In [54]:
response = await chain.ainvoke({"topic": "bears"})
response

'Why did the bear bring a flashlight to the party? \n\nBecause he wanted to be the "light" of the party!'